Fake News Detection

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [77]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')

In [78]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [79]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [80]:
data_fake["class"] = 0
data_true["class"] = 1

In [81]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [82]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis = 0, inplace = True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis = 0, inplace = True)

In [83]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [84]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\HP\AppData\Local\Temp\ipykernel_24824\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\HP\AppData\Local\Temp\ipykernel_24824\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [85]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [86]:
data_true_manual_testing.tail()

,title,text,subject,date,class
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [87]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [88]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [89]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [90]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [91]:
data = data.sample(frac = 1)

In [92]:
data.head()

,text,class
15314,The liberal pundits are pretty frustrated by t...,0
6309,"Throughout his presidency, President Obama has...",0
11724,https://www.youtube.com/watch?v=rUr8pYr5AXs,0
7805,"EDISON, N.J. (Reuters) - U.S. Republican presi...",1
10036,The lefty lunatic of the day just can t keep h...,0


In [93]:
data.reset_index(inplace= True)
data.drop(['index'], axis = 1, inplace = True)

In [94]:
data.columns

Index(['text', 'class'], dtype='object')

In [95]:
data.head()

,text,class
0,The liberal pundits are pretty frustrated by t...,0
1,"Throughout his presidency, President Obama has...",0
2,https://www.youtube.com/watch?v=rUr8pYr5AXs,0
3,"EDISON, N.J. (Reuters) - U.S. Republican presi...",1
4,The lefty lunatic of the day just can t keep h...,0


In [96]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [97]:
data['text'] = data['text'].apply(wordopt)

In [98]:
x = data['text']
y = data['class']

In [99]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=42)

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer(max_features=3000,
                                stop_words='english')
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [101]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=1000,
    solver='saga',
    n_jobs=-1  )
LR.fit(xv_train, y_train)

LogisticRegression(max_iter=1000, n_jobs=-1, solver='saga')

In [102]:
pred_lr = LR.predict(xv_test)

In [103]:
LR.score(xv_test, y_test)

0.9880793226381461

In [104]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4668
           1       0.98      0.99      0.99      4308

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [105]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier(max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5)
DT.fit(xv_train, y_train)

DecisionTreeClassifier(max_depth=20, min_samples_leaf=5, min_samples_split=10)

In [106]:
pred_dt = DT.predict(xv_test)

In [107]:
DT.score(xv_test, y_test)

0.9959893048128342

In [108]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4668
           1       0.98      0.99      0.99      4308

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [109]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state= 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [110]:
pred_gb = GB.predict(xv_test)

In [111]:
GB.score(xv_test, y_test)

0.9957664884135472

In [112]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4668
           1       0.98      0.99      0.99      4308

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [113]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [114]:
pred_rf = RF.predict(xv_test)

In [115]:
RF.score(xv_test, y_test)

0.9976604278074866

In [120]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4668
           1       1.00      1.00      1.00      4308

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



In [128]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Note A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_GB[0]),
                                                                                                              output_lable(pred_RF[0]),
                                                                                                              output_lable(pred_RF[0])))

In [129]:
news = str(input())
manual_testing(news)

 Sunday morning, after what must have seemed like another interminable weekend of scrutiny, terror, and being made to look like a fool, Donald Trump returned to doing what he does best: Being an absolute garbage person. He was still recovering from the humiliation of his hilariously bad attempt at pretending TIME Magazine  offered  him their Person of the Year honorific, followed by the embarrassment of trying to troll CNN, only to see their response tweet get twice the likes and retweets of his original.So Trump had to up his game. And how could he possibly piss off more people than by pivoting back to Roy Moore, the accused pedophile running for the US Senate in Alabama?Even some of Trump s most ardent supporters have begged the Doddering Dotard to back off his support for the toxic candidate. It s hard, as you might imagine, to truly get behind a guy who s accused of molesting young girls, especially with so much evidence against him. But Trump tried nonetheless:The last thing we ne



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


In [130]:
news = str(input())
manual_testing(news)

  KING OF PRUSSIA, Pennsylvania/WASHINGTON (Reuters) - In the Fox & Hound sports bar, next to a shopping mall in suburban Philadelphia, four Democrats are giving speeches to potential voters as they begin their journey to try to unseat Republican congressman Pat Meehan in next yearâ€™s elections. Winning this congressional district - Pennsylvaniaâ€™s 7th - is key to Democratsâ€™ hopes of gaining the 24 seats they need to retake the U.S. House of Representatives next November. The stakes are high - control of the House would allow them to block President Donald Trumpâ€™s legislative agenda. On the surface, Democrats face a significant hurdle. In nearly two-thirds of 34 Republican-held districts that are top of the partyâ€™s target list, household income or job growth, and often both, have risen faster than state and national averages over the past two years, according to a Reuters analysis of census data.Â (Graphic:Â tmsnrt.rs/2Bgq29K) That is potentially vote-winning news for Republica



LR Prediction: Note A Fake News 
DT Prediction: Note A Fake News 
GBC Prediction: Note A Fake News 
RFC Prediction: Note A Fake News
